In [67]:
import torch
from neural_poisson.model.encoder import MLP
import time

start_event = torch.cuda.Event(enable_timing=True)
end_event = torch.cuda.Event(enable_timing=True)

x = torch.randn((10_000, 3), device="cuda")
mlp = MLP(in_dim=3, out_dim=1, hidden_dim=256, num_layers=8).to("cuda") 
ts = []
for _ in range(1):
    # torch.cuda.synchronize()
    s = time.time()
    start_event.record()
    mlp(x)
    mlp(x)
    mlp(x)
    # torch.cuda.synchronize()
    e = time.time() - s
    end_event.record()
    ts.append(e)
print(sum(ts))
time.sleep(1)
elapsed_time = start_event.elapsed_time(end_event)  # Gives time in ms
print(f"Execution time: {elapsed_time:.3f} ms")


0.002025127410888672
Execution time: 6.956 ms


In [ ]:
x = torch.randn((10_000, 3), device="cuda")
mlp = MLP(in_dim=3, out_dim=1, hidden_dim=256, num_layers=8).to("cuda") 
ts = []
for _ in range(100):
    torch.cuda.synchronize()
    s = time.time()
    xs = torch.stack([x, x, x])
    mlp(xs)
torch.cuda.synchronize()
    e = time.time() - s
    ts.append(e)
print(sum(ts))

0.6201214790344238


In [ ]:
import torch
import torch.nn as nn
from neural_poisson.model.encoder import MLP

device = "cuda"
N = 1_000
points = torch.rand((N, 3), requires_grad=True).to("cuda")
mlp = MLP().to("cuda")
x = mlp(points)

_x = 0.5 * (x**2).sum()
# _x = x.sum()

# we want to compute dX/dp, which is the gradient of the estimated indicator function 
# X w.r.t the input points. However we can only compute dL/dp which computes the 
# gradients from a loss scalar. The chain rule is dL/dp = dL/dX * dX/dp. In order to 
# compute dX/dp we need to define the loss function do get dL/dX = 1, which results in
# a simple summation of dX, e.g. L=X.sum(), where the derivatives are 1.
dX = torch.autograd.grad(x.sum(), points)[0]

tensor([[ 0.0010,  0.0030, -0.0015],
        [ 0.0024,  0.0007, -0.0003],
        [ 0.0011,  0.0018, -0.0024],
        ...,
        [ 0.0023,  0.0013, -0.0015],
        [ 0.0008,  0.0018,  0.0003],
        [ 0.0001,  0.0016, -0.0007]], device='cuda:0')

In [78]:
x = time.time()
p = mlp(points)
x = time.time() - x
print(x)

0.0027997493743896484


In [73]:
torch.linalg.vector_norm(points, dim=-1).mean()

tensor(0.9457, device='cuda:0', grad_fn=<MeanBackward0>)

In [ ]:
from lightning.pytorch.loggers import WandbLogger

tensor([[-7.3827e-07, -6.2139e-06,  2.7392e-06],
        [ 1.6456e-06, -6.4065e-06,  1.2606e-06],
        [ 3.8234e-07, -2.8763e-06,  2.7467e-06],
        ...,
        [-2.3553e-06, -5.2600e-07,  3.0943e-07],
        [-1.6997e-06, -1.5128e-06,  3.3624e-06],
        [ 5.2575e-07, -4.5885e-06,  2.0370e-06]], device='cuda:0')

In [9]:
from typing import Any

import torch.nn as nn

from neural_poisson.model.activation import activation_fn

class MLP(nn.Sequential):
    def __init__(
        self,
        in_dim: int = 3,
        out_dim: int = 1,
        hidden_dim: int = 256,
        num_layers: int = 5,
        activation: str = "relu",
    ):
        layers: list[Any] = []

        # input layers
        layers.append(nn.Linear(in_dim, hidden_dim))
        layers.append(activation_fn(activation))

        # hidden layers
        for _ in range(num_layers):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(activation_fn(activation))

        # output layer
        layers.append(nn.Linear(hidden_dim, out_dim))

        # self.mlp = nn.Sequential(*layers)
        super().__init__(*layers)

    def forward(self, x):
        return super().forward(x)

import torch
mlp = MLP()


torch.Size([100, 1])

13